In [1]:
import os
import sys
import time
from datetime import datetime
import pandas as pd
from os import path
import yaml
from experiments import execute_command_on_server_and_clients, concat_dict

from execo import SshProcess
from execo_g5k import oarsub, oardel, OarSubmission, get_current_oar_jobs, get_oar_job_nodes, get_oar_job_info, Deployment, deploy

DEBUG - 2024-01-26 15:40:08,681 - jtop.py - 133 : Loaded jetson_variables variables
DEBUG - 2024-01-26 15:40:08,650 - jtop.py - 133 : Loaded jetson_variables variables
DEBUG - 2024-01-26 15:40:08,690 - jtop.py - 133 : Loaded jetson_variables variables
INFO - 2024-01-26 15:40:08,936 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 15:40:08,937 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp/energy.csv
INFO - 2024-01-26 15:40:08,912 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 15:40:08,913 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp/energy.csv
INFO - 2024-01-26 15:40:08,944 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 15:40:08,946 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp/energy.csv


In [5]:
jobname="fl-measure"
nodecount=3
walltime="03:00:00"
resources_selection="-t exotic -p estats"
site="toulouse"
force_redeploy=False
environment_dsc_file='./images/fl_jetson_image.yaml'

repository_dir = "/home/mjay/FL-G5K-Test/"
tmp_dir = "/tmp/"
jetson_sensor_monitor = f"{repository_dir}/jetson_monitoring_energy.py"
result_energy_file = "energy.csv"
log_file = "logs.log"
exp_csv = f"{repository_dir}/outputs/experiment_summary.csv"

# Reserve a job and deploy the chosen environment

In [6]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname,
                            queue='testing')
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 448456


In [7]:
nodes = get_oar_job_nodes(jobid, site)
nodes.sort(key=lambda n: n.address)
nodes

[Host('estats-10.toulouse.grid5000.fr'),
 Host('estats-5.toulouse.grid5000.fr'),
 Host('estats-8.toulouse.grid5000.fr')]

In [8]:
server=nodes[0]
clients=nodes[1:]
print("Server:{} \n Clients: {}".format(server,clients))

Server:Host('estats-10.toulouse.grid5000.fr') 
 Clients: [Host('estats-5.toulouse.grid5000.fr'), Host('estats-8.toulouse.grid5000.fr')]


In [9]:
deployment = Deployment(hosts=nodes, env_file=os.path.abspath(environment_dsc_file))
deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])
print("Deployement status:\n* ok: {}\n* failed: {}".format(deploy_ok, deploy_failed))

toulouse: Warning: Permanently added 'toulouse.grid5000.fr,172.16.127.110' (ECDSA) to the list of known hosts.toulouse: 

toulouse: Deployment #D-e1c652ad-dc77-4117-874c-fe6b37486604 startedtoulouse: toulouse: Grab the key file /home/mjay/.ssh/authorized_keystoulouse: toulouse: Grab the tarball file http://public.toulouse.grid5000.fr/~tunguyen/fl_jetson_image.tar.zsttoulouse: toulouse: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgztoulouse: toulouse: Using classical reboot instead of kexec (SetDeploymentMiniOS) because the last deployment is not trustedtoulouse: toulouse: Launching a deployment of fl_jetson_image:mjay:aarch64:2024011717 on estats-[5,8,10].toulouse.grid5000.frtoulouse: toulouse: Performing a Deploy[SetDeploymentMiniOSUntrusted] steptoulouse: toulouse:   switch_pxetoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] NODE: estats-5.toulouse.grid5000.frtoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] COMMAND: KADEPLOY_BOOTLOADER_NO_GRUB_FROM_DEST="1" KADEPLOY_CLUSTER="estats" KADEPLOY_VERSION="3.8.2.stable" KAD

toulouse: Connection to toulouse.grid5000.fr closed.toulouse: Warning: Permanently added 'estats-10.toulouse.grid5000.fr,172.16.121.10' (ECDSA) to the list of known hosts.


Deployement status:
* ok: {'estats-8.toulouse.grid5000.fr', 'estats-5.toulouse.grid5000.fr', 'estats-10.toulouse.grid5000.fr'}
* failed: set()


# Defining hyper parameters

In [65]:
exp_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

result_folder = f"{repository_dir}/outputs/{exp_datetime}/"
!mkdir -p $result_folder 
tmp_result_folder = f"{tmp_dir}/{exp_datetime}/"

# Create hyperparameters folder
hyperparams = {}
hyperparams["result_folder"]=result_folder
hyperparams["result_energy_file"]=result_energy_file
hyperparams["log_file"]=log_file
hyperparams["tmp_result_folder"]=tmp_result_folder
hyperparams["exp_datetime"]=exp_datetime
hyperparams["sleep_duration"]=30
hyperparams["timestamps"]={}

In [66]:
hyperparams["server"] = server.address
for cid in range(len(clients)):
    hyperparams[f"client_{cid}"] = clients[cid].address

In [67]:
path_to_yaml = repository_dir + "/config/config_file.yaml"
with open(path_to_yaml) as yaml_file:
    yaml_contents = yaml.load(yaml_file, Loader=yaml.FullLoader)
hyperparams.update(yaml_contents)

In [68]:
hyperparams

{'result_folder': '/home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17/',
 'result_energy_file': 'energy.csv',
 'log_file': 'logs.log',
 'tmp_result_folder': '/tmp//2024-01-26_16-16-17/',
 'exp_datetime': '2024-01-26_16-16-17',
 'sleep_duration': 30,
 'timestamps': {},
 'server': 'estats-10.toulouse.grid5000.fr',
 'client_0': 'estats-5.toulouse.grid5000.fr',
 'client_1': 'estats-8.toulouse.grid5000.fr',
 'params': {'num_classes': 10,
  'num_rounds': 1,
  'num_clients': 2,
  'batch_size': 128,
  'validation_split': 10,
  'num_clients_per_round_fit': 2,
  'num_clients_per_round_eval': 2,
  'root_data': '${hydra:runtime.cwd}/data'},
 'client': {'lr': 0.1,
  'local_epochs': 1,
  'cid': 0,
  'dry_run': False,
  'decay_rate': 0.1,
  'decay_steps': 10},
 'comm': {'host': '172.16.121.1', 'port': 8080},
 'defaults': [{'neuralnet': 'MobileNetV3Small'},
  {'strategy': 'fedavg'},
  {'optimizer': 'SGD'},
  '_self_'],
 'hydra': {'run': {'dir': './outputs/${hydra.job.name}_${client.cid}/${now:%Y-%m-%

# RUN EXPERIMENT

First step is to get to IP adress of the server so the clients can connect to it. To get it, we send a SshProcess to the server:

In [69]:
def get_host_ip(hostname):
    command = f"hostname -I"
    process = SshProcess(command, host=hostname)
    process.run()
    if process.ok:
        ip_address = process.stdout.strip()
        process.kill()
        return ip_address
    else:
        process.kill()
        return f"Failed to get IP for {hostname}"

# Example usage
hostname = "your_host"
ip_address_full = get_host_ip(server)
ip_address = ip_address_full.split(" ")[0]
print(f"IP address for {hostname} is {ip_address}")
hyperparams["comm"]["host"]=ip_address

IP address for your_host is 172.16.121.10


Now we define the SshProcess for the server and the clients, providing them with the hyper parameters of this experiment as defined in the previous section.
We first save results locally (in the /tmp/ folder) to reduce communication overhead. Results are copied back to the home dirs at the end of the training.

In [70]:
command = f"mkdir -p {tmp_result_folder}; echo -n > {tmp_result_folder}/logs.log"
_ = execute_command_on_server_and_clients(nodes, command, f"{repository_dir}/outputs/logs.log")

Successfully executed on Host('estats-10.toulouse.grid5000.fr') command 'mkdir -p /tmp//2024-01-26_16-16-17/; echo -n > /tmp//2024-01-26_16-16-17//logs.log'
Successfully executed on Host('estats-5.toulouse.grid5000.fr') command 'mkdir -p /tmp//2024-01-26_16-16-17/; echo -n > /tmp//2024-01-26_16-16-17//logs.log'
Successfully executed on Host('estats-8.toulouse.grid5000.fr') command 'mkdir -p /tmp//2024-01-26_16-16-17/; echo -n > /tmp//2024-01-26_16-16-17//logs.log'


In [71]:
command = f"cd {repository_dir}; \
    python3 main_server.py \
    hydra.run.dir={tmp_result_folder} \
    comm.host={ip_address}"
print(command)
run_server = SshProcess(
    command, 
    host=server, 
    connection_params={'user':'root'}, 
    stdout_handlers=[sys.stdout, f"{repository_dir}/outputs/logs.log"], 
    stderr_handlers=[sys.stderr, f"{repository_dir}/outputs/logs.log"]
    )

cd /home/mjay/FL-G5K-Test/;     python3 main_server.py     hydra.run.dir=/tmp//2024-01-26_16-16-17/     comm.host=172.16.121.10


In [72]:
run_clients = []
for (host,cid) in zip(clients,range(len(clients))):
    command = f"cd {repository_dir}; python3 client.py client.cid={cid} hydra.run.dir={tmp_result_folder} comm.host={ip_address}"
    run_client=SshProcess(
        command, 
        host=server, 
        connection_params={'user':'root'},
        stdout_handlers=[sys.stdout, f"{repository_dir}/outputs/logs.log"], 
        stderr_handlers=[sys.stderr, f"{repository_dir}/outputs/logs.log"])
    run_clients.append(run_client)

We can start the server, wait a few seconds so that it's ready before starting the clients.

In [73]:
# start the monitoring
command = f"python3 {jetson_sensor_monitor} --log-dir {tmp_result_folder} --log-csv {result_energy_file}"
jtop_processes = execute_command_on_server_and_clients(nodes, command, f"{repository_dir}/outputs/logs.log", background=True)
# sleep
hyperparams["timestamps"]["start_experiment_before_sleep"]=time.time()
time.sleep(hyperparams["sleep_duration"])
hyperparams["timestamps"]["start_experiment"]=time.time()
# start the server and the clients
run_server.start()
time.sleep(5)
for run_client in run_clients:
    run_client.start()

DEBUG - 2024-01-26 16:16:26,195 - jtop.py - 133 : Loaded jetson_variables variables
DEBUG - 2024-01-26 16:16:26,318 - jtop.py - 133 : Loaded jetson_variables variables
DEBUG - 2024-01-26 16:16:26,347 - jtop.py - 133 : Loaded jetson_variables variables
INFO - 2024-01-26 16:16:26,485 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 16:16:26,486 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp//2024-01-26_16-16-17/energy.csv
INFO - 2024-01-26 16:16:26,597 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 16:16:26,599 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp//2024-01-26_16-16-17/energy.csv
INFO - 2024-01-26 16:16:26,616 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-26 16:16:26,617 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp//2024-01-26_16-16-17/energy.csv


The server will disconnect once the training is done. So we wait until the server SshProcess is done to process results and start another experiment.

In [74]:
# wait until the training is done
run_server.wait()
# kill the monitoring
for proc in jtop_processes:
    proc.kill()
# copy the logs back to the home repository
cp_command = f"mkdir -p {result_folder}/server/; cp {tmp_result_folder}/* {result_folder}/server/" #; rm -r {tmp_result_folder}"
execute_command_on_server_and_clients([server], cp_command, f"{repository_dir}/outputs/logs.log")
for cid in range(len(clients)):
    cp_command = f"mkdir -p {result_folder}/client_{cid}/; cp {tmp_result_folder}/* {result_folder}/client_{cid}/" #; rm -r {tmp_result_folder}"
    execute_command_on_server_and_clients([clients[cid]], cp_command, f"{repository_dir}/outputs/logs.log")
# save the timestamps and sleep to make sure the power goes down
hyperparams["timestamps"]["end_experiment"]=time.time()
time.sleep(hyperparams["sleep_duration"])
hyperparams["timestamps"]["end_experiment_after_sleep"]=time.time()
# tmp_result_folder

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
main_server.py:19: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="config", config_name="config_file")
Number of available GPUs: 1
GPU 1: Xavier
Number of available GPUs: 1
GPU 1: Xavier
/usr/local/lib/python3.8/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hyd

Connection to estats-10.toulouse.grid5000.fr closed.
Connection to estats-10.toulouse.grid5000.fr closed.
Connection to estats-10.toulouse.grid5000.fr closed.
Connection to estats-10.toulouse.grid5000.fr closed.
Connection to estats-5.toulouse.grid5000.fr closed.
Connection to estats-8.toulouse.grid5000.fr closed.


Successfully executed on Host('estats-10.toulouse.grid5000.fr') command 'mkdir -p /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//server/; cp /tmp//2024-01-26_16-16-17//* /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//server/; rm -r /tmp//2024-01-26_16-16-17/'
Successfully executed on Host('estats-5.toulouse.grid5000.fr') command 'mkdir -p /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//client_0/; cp /tmp//2024-01-26_16-16-17//* /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//client_0/; rm -r /tmp//2024-01-26_16-16-17/'
Successfully executed on Host('estats-8.toulouse.grid5000.fr') command 'mkdir -p /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//client_1/; cp /tmp//2024-01-26_16-16-17//* /home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17//client_1/; rm -r /tmp//2024-01-26_16-16-17/'


In [75]:
hyperparams

{'result_folder': '/home/mjay/FL-G5K-Test//outputs/2024-01-26_16-16-17/',
 'result_energy_file': 'energy.csv',
 'log_file': 'logs.log',
 'tmp_result_folder': '/tmp//2024-01-26_16-16-17/',
 'exp_datetime': '2024-01-26_16-16-17',
 'sleep_duration': 30,
 'timestamps': {'start_experiment_before_sleep': 1706282185.3896494,
  'start_experiment': 1706282215.4203122,
  'end_experiment': 1706282296.7655945,
  'end_experiment_after_sleep': 1706282326.7960048},
 'server': 'estats-10.toulouse.grid5000.fr',
 'client_0': 'estats-5.toulouse.grid5000.fr',
 'client_1': 'estats-8.toulouse.grid5000.fr',
 'params': {'num_classes': 10,
  'num_rounds': 1,
  'num_clients': 2,
  'batch_size': 128,
  'validation_split': 10,
  'num_clients_per_round_fit': 2,
  'num_clients_per_round_eval': 2,
  'root_data': '${hydra:runtime.cwd}/data'},
 'client': {'lr': 0.1,
  'local_epochs': 1,
  'cid': 0,
  'dry_run': False,
  'decay_rate': 0.1,
  'decay_steps': 10},
 'comm': {'host': '172.16.121.10', 'port': 8080},
 'defaul

# Process results and save it as csv

In [76]:
def concat_dict(dict_list):
    new_dict = {}
    for d in dict_list:
        if isinstance(d, (dict)):
            new_dict.update(d)
    return new_dict

res = concat_dict(hyperparams["defaults"])
hyperparams["defaults"] = res

hyperparams_normalized = pd.json_normalize(hyperparams)
hyperparams_normalized.T

,0
result_folder,/home/mjay/FL-G5K-Test//outputs/2024-01-26_16-...
result_energy_file,energy.csv
log_file,logs.log
tmp_result_folder,/tmp//2024-01-26_16-16-17/
exp_datetime,2024-01-26_16-16-17
sleep_duration,30
server,estats-10.toulouse.grid5000.fr
client_0,estats-5.toulouse.grid5000.fr
client_1,estats-8.toulouse.grid5000.fr
timestamps.start_experiment_before_sleep,1706282185.389649


In [77]:
if path.exists(exp_csv):
    df = pd.read_csv(exp_csv)
    df = pd.concat([df, hyperparams_normalized], ignore_index=True)
else:
    df = pd.DataFrame(hyperparams_normalized)
df.to_csv(exp_csv, index=False)

If something went wrong:

In [ ]:
run_server.kill()
[run_client.kill() for run_client in run_clients]

# Kill the job once all the experiments are done.

In [ ]:
oardel([(jobid,site)])